In [339]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [340]:
ReelView = pd.read_csv('/content/drive/MyDrive/csv files/ReelView.csv')
movies_metadata = pd.read_csv('/content/drive/MyDrive/csv files/movies_metadata.csv')
keywords = pd.read_csv('/content/drive/MyDrive/csv files/keywords.csv')
links = pd.read_csv('/content/drive/MyDrive/csv files/links.csv')
links_small = pd.read_csv('/content/drive/MyDrive/csv files/links_small.csv')
ratings_small = pd.read_csv('/content/drive/MyDrive/csv files/ratings_small.csv')
box_office = pd.read_csv('/content/drive/MyDrive/csv files/boxoffice.csv')
BestBooks = pd.read_csv('/content/drive/MyDrive/csv files/books_1.Best_Books_Ever.csv')
best_selling_books = pd.read_csv('/content/drive/MyDrive/csv files/best-selling-books.csv')
ratings = pd.read_csv('/content/drive/MyDrive/csv files/ratings.csv')
author = pd.read_csv('/content/drive/MyDrive/csv files/author_info.csv')

<ipython-input-340-c18f3cb1a0f9>:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('/content/drive/MyDrive/csv files/movies_metadata.csv')


**Book Datasets**
1.   **BestBooks**: This dataset provides comprehensive information about 52478 books on Goodreads' Best Books Ever list, including ratings, genres, awards, and more
2.   **best_selling_books**: This dataset contains lists of best-selling books and book series in any language.


**Movie Datasets**
1.   **ReelView**: movie reviews from ReelView
2.   **movies_metadata**: The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.
3. **keywords**: Contains the movie plot keywords for our MovieLens movies
4. **links**: The file that contains the TMDB and IMDB IDs of all the movies featured in the Full MovieLens dataset.
5. **links_small**: Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.
6. **ratings_small**: The subset of 100,000 ratings from 700 users on 9,000 movies.


In [341]:
#adding a number of awards column into BestBooks dataframe
#dropping duplicates of the same book and movie
#adding suffix to each frame so we can keep track on what columns are about the book and what about the movie
BestBooks['award list'] = BestBooks['awards'].str.split(',')
BestBooks['num book awards'] = BestBooks['award list'].str.len()

BestBooks = BestBooks.drop_duplicates(subset='title')
movies_metadata = movies_metadata.drop_duplicates(subset='title')

BestBooks.columns = ['book_' + col if col != 'title' else col for col in BestBooks.columns]
movies_metadata.columns = ['movie_' + col if col != 'title' else col for col in movies_metadata.columns]

In [342]:

#merging the books and movies
#removing all columns that are not good for our model
#replace NaN values with appropriate values
#making sure the publish date type is coherent
#dropped all NaN rows since they were only 16 of them

df = pd.merge(BestBooks, movies_metadata, on='title')

df['book_firstPublishDate'] = df['book_firstPublishDate'].fillna(df['book_publishDate'])
df['book_pages'] = df['book_pages'].str.extract('(\d+)')
df['book_pages'] = df['book_pages'].fillna(np.median(df['book_pages'].dropna().astype(int)))
df['book_publish_date'] = pd.to_datetime(df['book_firstPublishDate'], errors='coerce', exact=False)

df = df.drop(columns=['book_bookId', 'book_series', 'book_description', 'book_isbn', 'book_language', 'book_edition',
                      'movie_production_countries', 'book_characters', 'book_bookFormat', 'movie_original_title',
                      'movie_spoken_languages', 'movie_status', 'movie_video', 'movie_tagline', 'movie_overview',
                      'movie_poster_path', 'movie_production_companies', 'movie_homepage', 'movie_id',
                      'book_coverImg', 'book_bbeScore', 'book_bbeVotes', 'book_price', 'book_award list',
                      'movie_adult', 'movie_belongs_to_collection', 'movie_original_language', 'movie_popularity',
                      'movie_runtime', 'book_publisher', 'book_publishDate', 'book_firstPublishDate'], axis=1)
df = df.dropna()

In [343]:
#Making sure the book came out before the movie
df = df[df['movie_release_date'] > df['book_publish_date'] + pd.DateOffset(years=1)]

In [344]:
#adding lifetime gross and studio
df = pd.merge(df, box_office[['title', 'studio' , 'lifetime_gross']], on='title', how='left')

In [345]:
df = df.dropna()

In [346]:
#adding author information such as workcount, fancount and gender

df['book_author'] = df['book_author'].str.lower()
df['book_author'] = df['book_author'].str.split(',').str[0]
df['book_author'] = df['book_author'].str.replace(r"\(.*\)", "", regex=True).str.strip()
df['book_author'] = df['book_author'].astype(str)

author['name'] = author['name'].str.lower()
author['name'] = author['name'].str.split(',').str[0]
author['name'] = author['name'].str.replace(r"\(.*\)", "", regex=True).str.strip()
author['name'] = author['name'].astype(str)

df = pd.merge(df, author[['name', 'workcount', 'fan_count']], left_on='book_author', right_on='name', how='left')

In [347]:
#filling NaN values

df['author_workcount'] = df['workcount'].fillna(df['workcount'].mean())
df['author_fan_count'] = df['fan_count'].fillna(df['fan_count'].mean())
df = df.drop(columns=['name', 'workcount', 'fan_count'], axis=1)

In [348]:
#creating a column with the quantile for each value in the numerical columns
col_names = df.quantile(np.arange(.01, 1.01, .01)).columns
quantiles = df.quantile(np.arange(0.01, 1.01, 0.01))
# Iterate through columns
for col in col_names:
    success = []
    # Iterate through elements in the column
    for a in df[col].values:
        # Find the index of the first quantile where the element is smaller
        index = (quantiles[col] > a).idxmax()
        success.append(index)
    # Create a new column with success values
    df['success_' + col] = success

<ipython-input-348-3d27fe7da933>:2: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  col_names = df.quantile(np.arange(.01, 1.01, .01)).columns
<ipython-input-348-3d27fe7da933>:3: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  quantiles = df.quantile(np.arange(0.01, 1.01, 0.01))


In [349]:
df.columns

Index(['title', 'book_author', 'book_rating', 'book_genres', 'book_pages',
       'book_awards', 'book_numRatings', 'book_ratingsByStars',
       'book_likedPercent', 'book_setting', 'book_num book awards',
       'movie_budget', 'movie_genres', 'movie_imdb_id', 'movie_release_date',
       'movie_revenue', 'movie_vote_average', 'movie_vote_count',
       'book_publish_date', 'studio', 'lifetime_gross', 'author_workcount',
       'author_fan_count', 'success_book_rating', 'success_book_numRatings',
       'success_book_likedPercent', 'success_book_num book awards',
       'success_movie_revenue', 'success_movie_vote_average',
       'success_movie_vote_count', 'success_lifetime_gross',
       'success_author_workcount', 'success_author_fan_count'],
      dtype='object')

In [350]:
#making an average success metric of .01 through 1
book_success_cols = ['success_book_rating',
       'success_book_numRatings', 'success_book_likedPercent',
       'success_book_num book awards','success_author_fan_count',
       'success_author_workcount']

#removing success_movie_revenue since there are lots of 0s
movie_success_cols = ['success_movie_vote_average', 'success_movie_vote_count',
       'success_lifetime_gross']
#removing average vote of 0 (8 movies)
df = df[df['movie_vote_average']!=0]

#removing movies with no votes (8 movies)
df = df[df['movie_vote_count']!=0]

df['avg_book_success'] = df[book_success_cols].mean(axis=1)
df['avg_movie_success'] = df[movie_success_cols].mean(axis=1)

In [351]:

df.columns

Index(['title', 'book_author', 'book_rating', 'book_genres', 'book_pages',
       'book_awards', 'book_numRatings', 'book_ratingsByStars',
       'book_likedPercent', 'book_setting', 'book_num book awards',
       'movie_budget', 'movie_genres', 'movie_imdb_id', 'movie_release_date',
       'movie_revenue', 'movie_vote_average', 'movie_vote_count',
       'book_publish_date', 'studio', 'lifetime_gross', 'author_workcount',
       'author_fan_count', 'success_book_rating', 'success_book_numRatings',
       'success_book_likedPercent', 'success_book_num book awards',
       'success_movie_revenue', 'success_movie_vote_average',
       'success_movie_vote_count', 'success_lifetime_gross',
       'success_author_workcount', 'success_author_fan_count',
       'avg_book_success', 'avg_movie_success'],
      dtype='object')

In [352]:
#removing all columns not used for model
df = df.drop(columns=['movie_budget', 'movie_genres', 'movie_imdb_id', 'movie_release_date',
                'movie_revenue', 'movie_vote_average', 'movie_vote_count', 'lifetime_gross',
                 'success_book_rating', 'success_book_numRatings', 'success_book_likedPercent',
                 'success_book_num book awards', 'success_movie_revenue', 'success_movie_vote_average',
                'success_movie_vote_count', 'success_lifetime_gross', 'success_author_workcount',
                 'success_author_fan_count', 'studio', 'book_ratingsByStars', 'book_setting', 'book_awards',
                      'avg_book_success'], axis=1)

In [353]:
#add year column
df['year'] = df['book_publish_date'].dt.year
df = df.drop(['book_publish_date'], axis=1)
#encode author col
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Author Encoded'] = label_encoder.fit_transform(df['book_author'])
df = df.drop(['book_author'], axis=1)

In [354]:
#find most common genres and one hot encode them
from collections import Counter
import ast

df['book_genres'] = df['book_genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

genre_counts = Counter([genre for sublist in df['book_genres'] for genre in sublist])
genres_over_60 = {genre: count for genre, count in genre_counts.items() if count > 60}
most_common_genres = genres_over_60.keys()
df['genres'] = df['book_genres'].apply(lambda genres: [genre for genre in genres if genre in most_common_genres])
exploded_genres = df.explode('genres')

encoded_genres = pd.get_dummies(exploded_genres['genres'])
final_encoded_genres = encoded_genres.groupby(level=0).sum()
final_df = pd.concat([df, final_encoded_genres], axis=1)
df = final_df.drop(['genres', 'book_genres'], axis=1)
df.set_index('title', inplace=True)

In [355]:
most_common_genres

dict_keys(['Young Adult', 'Fiction', 'Fantasy', 'Science Fiction', 'Romance', 'Adventure', 'Childrens', 'Audiobook', 'Classics', 'Historical Fiction', 'Historical', 'Humor', 'Novels', 'Mystery', 'Thriller', 'Suspense', 'Mystery Thriller', 'Crime', 'Adult', 'Adult Fiction', 'Contemporary', 'Chick Lit', 'Drama', 'Literature', 'Nonfiction', 'Paranormal', 'Horror', 'Literary Fiction', 'American'])

In [356]:
df.rename(columns={'book_rating': 'Rating', 'book_pages': 'Pages',
                   'book_numRatings': 'Total Ratings', 'book_likedPercent': 'Percent Liked',
                   'book_num book awards': 'Nr Awards', 'author_workcount': 'Author Workcount',
                   'author_fan_count': 'Author Fancount', 'avg_movie_success': 'Movie Success'}, inplace=True)

In [361]:
df

,Rating,Pages,Total Ratings,Percent Liked,Nr Awards,Author Workcount,Author Fancount,Movie Success,year,Author Encoded,...,Mystery,Mystery Thriller,Nonfiction,Novels,Paranormal,Romance,Science Fiction,Suspense,Thriller,Young Adult
title,,,,,,,,,,,,,,,,,,,,,
The Hunger Games,4.33,374,6376780,96.0,41,32.0,81311.0,0.580000,2008,568,...,0,0,0,0,0,1,1,0,0,1
Harry Potter and the Order of the Phoenix,4.50,870,2507623,98.0,9,242.0,209174.0,0.963333,2003,227,...,0,0,0,0,0,0,0,0,0,1
The Book Thief,4.37,552,1834276,96.0,20,16.0,33743.0,0.806667,2005,395,...,0,0,0,0,0,0,0,0,0,1
The Hitchhiker's Guide to the Galaxy,4.22,193,1436325,94.0,1,103.0,19029.0,0.756667,1979,150,...,0,0,0,1,0,0,1,0,0,0
The Da Vinci Code,3.86,489,1933446,89.0,9,76.0,81601.0,0.826667,2003,113,...,1,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Risk,3.53,176,345,88.0,1,30.0,110.0,0.346667,2009,102,...,1,0,0,0,0,0,0,0,1,0
Savannah,4.17,608,2007,95.0,1,56.0,134.0,0.166667,1983,174,...,0,0,0,0,0,1,0,0,0,0
Last Orders,3.68,352,12451,89.0,4,41.0,429.0,0.303333,1996,198,...,0,0,0,1,0,0,0,0,0,0


In [360]:
pd.read_csv('/content/final_df.csv')

,title,Rating,Pages,Total Ratings,Percent Liked,Nr Awards,Author Workcount,Author Fancount,Movie Success,year,...,Mystery,Mystery Thriller,Nonfiction,Novels,Paranormal,Romance,Science Fiction,Suspense,Thriller,Young Adult
0,The Hunger Games,4.33,374.0,6376780,96.0,41,32.0,81311.0,0.580000,2008,...,0,0,0,0,0,1,1,0,0,1
1,Harry Potter and the Order of the Phoenix,4.50,870.0,2507623,98.0,9,242.0,209174.0,0.963333,2003,...,0,0,0,0,0,0,0,0,0,1
2,The Book Thief,4.37,552.0,1834276,96.0,20,16.0,33743.0,0.806667,2005,...,0,0,0,0,0,0,0,0,0,1
3,The Hitchhiker's Guide to the Galaxy,4.22,193.0,1436325,94.0,1,103.0,19029.0,0.756667,1979,...,0,0,0,1,0,0,1,0,0,0
4,The Da Vinci Code,3.86,489.0,1933446,89.0,9,76.0,81601.0,0.826667,2003,...,1,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,Risk,3.53,176.0,345,88.0,1,30.0,110.0,0.346667,2009,...,1,0,0,0,0,0,0,0,1,0
809,Savannah,4.17,608.0,2007,95.0,1,56.0,134.0,0.166667,1983,...,0,0,0,0,0,1,0,0,0,0
810,Last Orders,3.68,352.0,12451,89.0,4,41.0,429.0,0.303333,1996,...,0,0,0,1,0,0,0,0,0,0
811,The Wanderers,3.90,320.0,1640,94.0,1,87.0,614.0,0.413333,1974,...,1,0,0,1,0,0,0,0,0,0
